In [1]:
import pickle
import numpy as np
import itertools
from matplotlib import pyplot as plt

import sys
#path_to_codebase = '/mnt/Study/umass/sem3/maxEnt/src/codebase/'
path_to_codebase = './codebase/'
sys.path.insert(0, path_to_codebase)
from codebase.utils import load_disease_data
from codebase.extract_features import ExtractFeatures
from codebase.optimizer import Optimizer


In [2]:
obj1f = '../out/pickles/obj_red_mba_fillNA.pk'
with open(obj1f, 'rb') as of1:
    optobj = pickle.load(of1)
    solopt = pickle.load(of1)

In [3]:
obj_file = '../out/pickles/obj_red_mba.pk'
with open(obj_file, 'rb') as ofile:
    featobj1 = pickle.load(ofile)
    optobj1 = pickle.load(ofile)
    solopt1 = pickle.load(ofile)

In [4]:
for i in range(len(solopt[0])):
    print solopt[0][i][0]

print('\n')
    
for i in range(len(solopt1[0])):
    print solopt1[0][i][0]
    

[-10.42951701004439]
[-10.02403712815968]
[-4.838159841775679]
[-4.28625253 -4.11309536 -3.85165339 -4.14317337 -3.77791183 -3.98728892
 -3.35531745 -5.44290636 -5.02083616 -4.02506058 -4.87524806 -4.58353127
 -3.54131297 -3.46094673 -4.26340308  1.38454472  2.8699368   1.43021511
  2.28567615  3.21589774  0.70103747  1.87686461  0.9760629   0.75912796
  2.20023983  1.41818161  2.0682994   1.11654243  1.48312033  1.87598469
  1.4417148   0.2271023   0.77653704  1.29384824  0.91866381  1.423273
  2.06395804  2.11519042 -1.41504662 -1.09715426  0.41933468 -0.88210088
  4.41898059  0.42599233 -0.69356795 -0.89866245 -0.06010248 -0.79796062
  0.07975043  1.54988692  0.14934552 -0.86898263 -0.02867707 -0.94249605
  0.31534105  0.00783835 -0.1346633  -0.35347186 -0.11054583 -0.03236675
 -0.53263211 -1.11262205  0.74749183 -0.04211486 -0.40692872 -2.31196688
  0.91381707 -0.2643467  -0.1840477 ]
[-7.20993179758437]
[-5.816414992558676]
[-6.002229747145664]
[-4.6870334907829685]
[-4.3950475539

In [5]:
print len(solopt[0][3][0]), len(solopt1[0][3][0])

69 72


In [6]:
# normalization constants
for i in range(len(solopt[1])):
    diff = np.abs(solopt[1][i] - solopt1[1][i]) 
    print solopt[1][i], solopt1[1][i], diff 

1.0000295473348304 1.0000483126796629 1.8765344832472053e-05
1.000044321657039 1.0000724707701227 2.8149113083797417e-05
1.0079216176777153 1.0130178383536839 0.005096220675968599
1.4091827124498129 1.8223730136376277 0.41319030118781486
1.0007392075694854 1.001209219086314 0.00047001151682857234
1.0029782631243611 1.0048788776154183 0.001900614491057162
1.0024732313433942 1.0040502522312766 0.0015770208878824743
1.0092139790076335 1.0151541158873005 0.005940136879666991
1.0123382935766096 1.020333218317149 0.007994924740539311
1.0144167366023258 1.0237900932314465 0.009373356629120755
1.003216101255317 1.005269292409305 0.0020531911539880543
1.0006208609271523 1.0010155475493872 0.0003946866222348522
1.0038558505116417 1.0063200369479084 0.0024641864362666688
1.0015535991714137 1.002542742287015 0.0009891431156012764
1.0079516349991064 1.0130674171051022 0.005115782105995725
1.0002364275792772 1.0003866321919628 0.00015020461268555607
1.0003103341264095 1.0005075160713424 0.0001971819

In [7]:
optobj.feats_obj.feat_partitions

[[0],
 [1],
 [2],
 [3, 8, 32, 35, 4, 37, 13, 14, 15, 23, 28, 29, 30, 31, 36],
 [5],
 [6],
 [7],
 [9],
 [10],
 [11],
 [12],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21],
 [22],
 [24],
 [25],
 [26],
 [27],
 [33],
 [34]]

In [8]:
optobj1.feats_obj.feat_partitions

[[0],
 [1],
 [2],
 [3, 8, 32, 35, 4, 37, 13, 14, 15, 23, 28, 29, 30, 31, 36],
 [5],
 [6],
 [7],
 [9],
 [10],
 [11],
 [12],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21],
 [22],
 [24],
 [25],
 [26],
 [27],
 [33],
 [34]]

In [9]:
t = solopt[0]
print len(t), len(optobj.feats_obj.feat_partitions)

24 24


In [10]:
def prob_dist(optobj, rvec):
    """
    Function to compute the probability for a given input vector
    """
    logp_sum = 0.0
    prob_product = 1.0
    parts = optobj.feats_obj.feat_partitions
    solution = optobj.opt_sol
    norm_sol = optobj.norm_z

    # `partition` will be a set of indices in the i-th parition        
    for i,partition in enumerate(parts):
        tmpvec = rvec[partition]
        term_exp = optobj.compute_constraint_sum(solution[i][0], tmpvec, partition)
        prob_part = (1.0/norm_sol[i]) * np.exp(term_exp)
        prob_product *= prob_part
        logp_sum += np.log(prob_part)

#     return prob_product
#     return logp_sum
    return np.exp(logp_sum)

In [11]:
optobj.feats_obj.data_arr.shape[1]

38

In [12]:
x1 = np.zeros(38)
idx = [4,8,13,31]
x1[idx] = 1
print prob_dist(optobj, x1), prob_dist(optobj1, x1)

0.16400855600421596 0.028142573190359694


In [24]:
def compute_emp_prob(data, vec):
    count = 0.0
    for tmp in data:
        if sum(tmp == vec) == len(vec):
            count += 1
    count /= data.shape[0]
    return count
            

In [12]:
def constraint_counts(optobj):
    for k in optobj.feats_obj.two_way_dict.keys():
        

In [13]:
def iter_nCk_binvect(n, k):
    for bits in itertools.combinations(range(n), k):
        tmp = np.zeros(n)
#         print bits
        tmp[list(bits)] = 1
        yield tmp

In [15]:
def compute_prob_exact(optobj, k):
    probsum = 0
    maxi, v = 0, None
    numfeat = optobj.feats_obj.data_arr.shape[1]
    for vec in iter_nCk_binvect(numfeat, k):
        tmp = prob_dist(optobj, vec)
        probsum += tmp
        if tmp >= maxi:
            maxi = tmp
            v = vec
    print(maxi, v)
    return probsum

In [18]:
# compute_prob_exact(optobj, 4)
compute_prob_exact(optobj, 5)

(0.11553391666410365, array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
       0., 0., 0., 0.]))


0.6629886331225583

In [19]:
compute_prob_exact(optobj, 6)

(0.10523628291656087, array([0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
       0., 0., 0., 0.]))


0.5805951043629636

In [25]:
v6max = np.array([0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
       0., 0., 0., 0.])
print prob_dist(optobj, v6max)
datarr = optobj.feats_obj.data_arr
print compute_emp_prob(datarr, v6max)

0.10523628291656087
0.0


In [26]:
v0 = np.zeros(38)
print prob_dist(optobj, v0)
datarr = optobj.feats_obj.data_arr
print compute_emp_prob(datarr, v0)

0.543629684279562
0.609233269316


In [105]:
compute_prob_exact(optobj1, 4)

0.028142573190359694


0.14926281918497553

In [43]:
low = 1
hgh = 5

ans = np.zeros(hgh+1)

for i in range(low, hgh+1):
    ans[i] = compute_prob_exact(optobj, i)

In [62]:
ans[0] = optobj.prob_dist(np.zeros(38))
ans

NameError: name 'ans' is not defined

In [16]:
low = 1
hgh = 5

ans2 = np.zeros(hgh+1)

for i in range(low, hgh+1):
    ans2[i] = compute_prob_exact(optobj, i)

ans2[0] = optobj.prob_dist(np.zeros(38))
ans2

array([0.54362968, 0.2909475 , 0.10353954, 0.11417444, 0.39635492,
       0.66298863])

In [58]:
optobj.feats_obj.feat_partitions

[[0],
 [1],
 [2],
 [3, 8, 32, 35, 4, 37, 13, 14, 15, 23, 28, 29, 30, 31, 36],
 [5],
 [6],
 [7],
 [9],
 [10],
 [11],
 [12],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21],
 [22],
 [24],
 [25],
 [26],
 [27],
 [33],
 [34]]

In [116]:
data = optobj.feats_obj.data_arr
N, _ = data.shape
p = np.zeros(5)
m = np.zeros(5)

for i in range(N):
    v = np.sum(data[i], dtype=int)
    if v <= 4:
        m[v] += 1
        p[v] += optobj.prob_dist(data[i])

print m
print p
# m = m/N
# np.log(m/(1-m))

[41239. 14635.  6011.  2975.  1549.]
[2.24187445e+04 2.39689231e+02 6.76075423e+00 2.86210346e+00
 2.94618623e+00]


In [119]:
data = optobj.feats_obj.data_arr
N, _ = data.shape
x = np.zeros(38)
idx = [4,8,13,31]
x[idx] = 1

c = 0
for i in range(N):
    v = data[i]
    if sum(v==x) == 38:
        c += 1
print c
# m = m/N
# np.log(m/(1-m))

12


In [80]:
np.sum(data, axis=0, dtype=int)

array([   2,    3,  532, 1024, 3388,   50,  201,  167, 3649,  618,  825,
        962,  217, 5989,  484, 1133,   42,  260,  105,  534,   16,   21,
       4281, 1409, 1468, 1092, 1807, 2120,  629, 1066, 3234, 2805, 1985,
       1286, 1161, 1322, 1394, 1848])

In [84]:
x = np.zeros(38)
p = optobj.prob_dist(x)
p


0.543629684279562